# Desarrollo

## Diseño de acoples

El siguiente es el diseño para las microtiras que acoplarán el amplificador a $50 \Omega$.

In [2]:
from hammerstad import *
import numpy as np
import matplotlib.pyplot as plt
from adapt_imp import *

Se toma como referencia un $\epsilon_r = 4.15$ para un material FR4.

In [3]:
er = 4.15 # Relative permittivity
Zo = 50 # Ohm
H = 1.6e-3 # mm
t = 35e-6

Se comienza calculando el ratio $\frac{W}{H}$.

In [4]:
WH = get_WH(Zo,er)
WH

1.9967298570941503

Resta calcular el $W$ efectivo.

In [5]:
W = WH*H
W

0.0031947677713506407

In [6]:
We = get_W_efectivo(W,H,t)
We

0.0032562157540274713

Se obtiene $W_e=3.26 \ mm$. Ahora hay que buscar $\epsilon_r'$.

In [7]:
erf = get_permitividad_efectiva(WH,er)
erf

3.1698766519288357

Y finalmente corroborar $Z_0$.

In [8]:
z0 = get_impedancia_caracteristica(WH,erf)
z0

50.248434379498356

Se va a trabajar a una $f=1.8 \ GHz$.

In [9]:
lda = 300/1800
lda

0.16666666666666666

In [10]:
lde = lda/np.sqrt(erf)
lde

0.09361114732265245

Para la linea que une al generador con el amplificador, se tiene:

In [11]:
Zin = np.conj((6.82-8.03j))
Zout = np.conj((21.71+7.57j))

d1, Zl41 = s_L4_stub_in(Zin,50,Zo,lde)
print('Longitud del adaptador stub:',d1)
print('Impedancia caracteristica del transformador lambda/4:',Zl41)

Longitud del adaptador stub: 0.01938445602611267
Impedancia caracteristica del transformador lambda/4: 28.526020945230723


Ahora, para la salida del amplificador.

In [12]:
d2, Zl42 = out_L4_stub_l(Zout,50,Zo,lde)
print('Longitud del adaptador stub:',d2)
print('Impedancia caracteristica del transformador lambda/4:',Zl42)

Longitud del adaptador stub: 0.004998578261338278
Impedancia caracteristica del transformador lambda/4: 32.94692701907114


Ahora, se calculan los dos adaptadores de $\lambda /4$. Para eso se calculan como microtiras, en donde se usa la impedancia característica recién obtenida. Para el adaptador que va entre la entrada y el amplificador, se tiene:

In [15]:
WH1 = get_WH(Zl41,er)
W1 = WH1*H
We1 = get_W_efectivo(W1,H,t)
erf1 = get_permitividad_efectiva(WH1,er)
lde1 = lda/np.sqrt(erf1)
l1 = lde1/4
l1

0.02259235736571817

El ancho de la microtira entre el acople de entrada y el amplificador es:

In [17]:
W1

0.007293402083030762

Y su longitud de onda efectiva:

In [19]:
lde1

0.09036942946287269

De ser necesario, cada una de las microtiras calculadas puede alargarse en un factor $\lambda/2$, ya que se representa de esa forma la periodicidad en la carta de Smith. Ahora se sigue con la segunda microtira, la que aparece entre el amplificador y el acople de salida.

In [16]:
WH2 = get_WH(Zl42,er)
W2 = WH2*H
We2 = get_W_efectivo(W2,H,t)
erf2 = get_permitividad_efectiva(WH2,er)
lde2 = lda/np.sqrt(erf2)
l2 = lde2/4
l2

0.0227880988769765

El ancho es:

In [18]:
W2

0.005993255609842166

Y su longitud de onda efectiva:

In [20]:
lde2

0.091152395507906